# 🛠️ Project – Art-Tech Store AI Assistant

## 🛍️ Product Availability
- Check availability of **printers** and **printer papers** with:
  - Product name, price, brand, type (e.g., laser/inkjet), and stock status.
- Alerts user if a product is out of stock.

## 🧭 Guided Shopping Experience
- Guides users through:
  1. Choosing product category (printer or paper)
  2. Filtering options (brand, price range, type)
  3. Adding selected products to cart
- Ensures correct input for smooth shopping flow.

## 🧾 Receipt Generation
- Creates a unique receipt file: `customerName_orderNumber.txt`
- Receipt includes:
  - Customer name and contact
  - Product details (name, price, quantity)
  - Total cost and order summary

## 📦 Generate Order Summary Report
- Summarizes all purchases into a single file: `order_summary.txt`
- Useful for inventory and sales review

## 🎯 Product Recommendation
- Recommends:
  - Printers based on paper type, usage (home/office), or brand preference
  - Compatible paper based on selected printer

## 💬 Interactive Chat Interface
- Real-time conversation via **Gradio**
- Polite, helpful answers to product-related questions

## 🛠️ Modular Tool Support
- Integrated tools for:
  - Checking product availability
  - Adding to cart and generating receipts
  - Creating summary reports
- Easily extendable for:
  - Promotions
  - Customer reviews
  - Delivery tracking

## 🛡️ Error Handling
- Validates user inputs (e.g., product name, quantity)
- Graceful messages to guide user and prevent mistakes


In [ ]:
import os
import json
import random
from dotenv import load_dotenv
import gradio as gr
from openai import OpenAI

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

# MODEL = "gpt-4o-mini"
MODEL = "gpt-3.5-turbo"
openai = OpenAI()

In [ ]:
# Using local LLM (that can't even handle basic greeting like Hi!!

# MODEL = "llama3.2"
# openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [ ]:
###############################################################################
# 1) System Prompt
###############################################################################
system_message = (
    "You are a helpful assistant for an online store called art-tech.store that sells printers and printer papers.\n\n"
    "When the user wants to purchase a product, follow these steps:\n"
    "1. Ask whether they are interested in printers or printer papers.\n"
    "2. Ask for filtering preferences (e.g., brand, price range, type).\n"
    "3. Call the function 'check_product_availability' with the selected category and filters.\n"
    "   - If it returns an empty list, say: 'No products found for your selection.'\n"
    "   - If it returns products, list them EXACTLY, in a numbered list, showing name, price, brand, and availability.\n"
    "4. Wait for the user to select a product by number and quantity.\n"
    "5. Ask for customer first name and contact info.\n"
    "6. Then call 'add_to_cart_and_generate_receipt' to confirm and show the user the receipt and order details.\n\n"
    "You also have a tool 'generate_report' which summarizes ALL purchases in a single file.\n\n"
    "IMPORTANT:\n"
    "- Always call 'check_product_availability' if user mentions a new category or changes filters.\n"
    "- Do not invent products or details. Use only what the function calls return.\n"
    "- Every time an order is placed, produce a new receipt file named customerName_orderNumber.txt.\n"
    "- If no matching products are found, say so.\n"
    "- If the user wants a full order summary, call 'generate_report' with no arguments.\n"
    "If you don't know something, say so.\n"
    "Keep answers short and courteous.\n"
)


In [ ]:
###############################################################################
# 2) Mini Printer Availability with Price & Stock
###############################################################################
product_availability = {
    "mini_printers": [
        {
            "name": "Phomemo M110 מדפסת מדבקות חכמה",
            "brand": "Phomemo",
            "price": "₪300",  # Update if pricing is known
            "type": "Label Printer",
            "availability": "360,745 in stock (24 variants)"
        },
        {
            "name": "Niimbot B1 Label Printer",
            "brand": "Niimbot",
            "price": "₪350",
            "type": "Portable Thermal Label Printer",
            "availability": "13,029 in stock (18 variants)"
        },
        {
            "name": "Niimbot B21 Mini Portable Thermal Label Printer",
            "brand": "Niimbot",
            "price": "₪500",
            "type": "Adhesive Sticker Printer",
            "availability": "141 in stock (12 variants)"
        },
        {
            "name": "Dolewa D3 Portable Mini Printer",
            "brand": "Dolewa",
            "price": "₪450",
            "type": "Thermal Photo & Label Printer",
            "availability": "336 in stock (6 variants)"
        },
        {
            "name": "PrintPro Mini מדפסת כיס חכמה",
            "brand": "PrintPro",
            "price": "₪550",
            "type": "Mini Pocket Printer",
            "availability": "336 in stock (6 variants)"
        },
        {
            "name": "מיני מדפסת טרמית מעוצבת לילדים",
            "brand": "Art-Tech",
            "price": "₪200",
            "type": "Kids Thermal Printer",
            "availability": "62 in stock (11 variants)"
        },
        {
            "name": "Children Digital Camera Instant Print",
            "brand": "Art-Tech",
            "price": "₪250",
            "type": "Photo Printing Camera with 32G Memory Card",
            "availability": "160 in stock (3 variants)"
        }
    ],
    "mini_printer_papers": [
        {
            "name": "HP Printer Paper 8.5x11, 500 Sheets",  # example only
            "brand": "HP",
            "price": "$9.99",
            "type": "Standard",
            "availability": "In stock"
        },
        {
            "name": "Mini Printer Paper 57*25mm Color Sticker",
            "brand": "Art-Tech",
            "price": "₪70",
            "type": "Self-adhesive Color Label Paper",
            "availability": "71,996 in stock (9 variants)"
        },
        {
            "name": "מדבקות שקופות למדפסת טרמית",
            "brand": "Art-Tech",
            "price": "₪55",
            "type": "Transparent Labels",
            "availability": "11,762 in stock (12 variants)"
        },
        {
            "name": "גלילי נייר מדבקה",
            "brand": "Art-Tech",
            "price": "₪40",
            "type": "Sticker Paper Rolls",
            "availability": "42 in stock (4 variants)"
        },
        {
            "name": "Niimbot B21/B1/B3S Thermal Label Sticker Paper",
            "brand": "Niimbot",
            "price": "₪55",
            "type": "Printable White Label Paper 20–50mm",
            "availability": "1,342 in stock (14 variants)"
        },
        {
            "name": "Mini Printer Sticker Paper 25X57mm",
            "brand": "Paperang-compatible",
            "price": "₪65",
            "type": "Color Self-Adhesive Thermal Rolls",
            "availability": "3,023 in stock (20 variants)"
        },
        {
            "name": "3/5/10 NiiMBOT White Label Paper Rolls",
            "brand": "Niimbot",
            "price": "₪40",
            "type": "Waterproof Self-adhesive Rolls",
            "availability": "1,400 in stock (9 variants)"
        }
    ]
}


In [ ]:
# A global list of flight bookings
flight_bookings = []


In [ ]:
###############################################################################
# 3) Helper Functions for Art-Tech Store
###############################################################################

product_orders = []

def check_product_availability(category: str, filters: dict = None):
    """
    Return list of products in the given category from 'product_availability'.
    Optionally filter by brand, type, etc.
    """
    print(f"[TOOL] check_product_availability({category}, {filters=})")
    category = category.lower()
    products = product_availability.get(category, [])
    
    if filters:
        for key, val in filters.items():
            products = [p for p in products if p.get(key, "").lower() == val.lower()]
    return products


def add_to_cart_and_generate_receipt(customer_name: str, contact: str, product: dict, quantity: int, order_number: int):
    """
    Create a text file: customerName_orderNumber.txt containing order details.
    """
    safe_name = customer_name.replace(" ", "_")
    filename = f"{safe_name}_{order_number}.txt"

    content = (
        "Art-Tech Store Receipt\n"
        "=======================\n"
        f"Order #     : {order_number}\n"
        f"Customer    : {customer_name}\n"
        f"Contact     : {contact}\n"
        f"Product     : {product['name']}\n"
        f"Brand       : {product['brand']}\n"
        f"Type        : {product.get('type', 'N/A')}\n"
        f"Price       : {product['price']}\n"
        f"Quantity    : {quantity}\n"
        f"Availability: {product['availability']}\n"
    )
    with open(filename, "w") as f:
        f.write(content)

    print(f"[TOOL] Receipt file generated => {filename}")
    return filename


def place_order(category, product_index, quantity, customer_name, contact_info):
    """
    Places an order for a product by index in the filtered list.
    """
    print(f"[TOOL] place_order({category=}, {product_index=}, {quantity=})")

    try:
        idx = int(product_index)
    except ValueError:
        return "Error: Product option number is not a valid integer."

    products = product_availability.get(category.lower(), [])
    if not products:
        return f"Error: No products found in category '{category}'."

    pick = idx - 1
    if pick < 0 or pick >= len(products):
        return f"Error: Invalid product option #{idx} for category '{category}'."

    selected_product = products[pick]

    order = {
        "category": category,
        "product": selected_product["name"],
        "brand": selected_product["brand"],
        "type": selected_product.get("type", ""),
        "price": selected_product["price"],
        "quantity": quantity,
        "customer_name": customer_name,
        "contact": contact_info,
    }
    product_orders.append(order)

    order_number = len(product_orders)
    receipt_filename = add_to_cart_and_generate_receipt(customer_name, contact_info, selected_product, quantity, order_number)

    confirmation = (
        f"Order #{order_number} confirmed for {customer_name}. "
        f"{selected_product['name']} x{quantity}. Receipt saved to {receipt_filename}."
    )
    print(f"[TOOL] {confirmation}")
    return confirmation


def generate_report():
    """
    Summarize ALL orders in a single file called order_summary.txt.
    """
    print(f"[TOOL] generate_report called.")

    report_content = "Art-Tech Store Order Summary Report\n"
    report_content += "===================================\n"

    if not product_orders:
        report_content += "No orders found.\n"
    else:
        for i, order in enumerate(product_orders, start=1):
            report_content += (
                f"Order #     : {i}\n"
                f"Customer    : {order['customer_name']}\n"
                f"Contact     : {order['contact']}\n"
                f"Product     : {order['product']}\n"
                f"Brand       : {order['brand']}\n"
                f"Type        : {order['type']}\n"
                f"Price       : {order['price']}\n"
                f"Quantity    : {order['quantity']}\n"
                "-------------------------\n"
            )

    filename = "order_summary.txt"
    with open(filename, "w") as f:
        f.write(report_content)

    msg = f"Summary report generated => {filename}"
    print(f"[TOOL] {msg}")
    return msg


In [ ]:
###############################################################################
# 4) Tools JSON Schemas for Art-Tech Store
###############################################################################
price_function = {
    "name": "get_product_price",
    "description": "Get the price of a product (not strictly needed now).",
    "parameters": {
        "type": "object",
        "properties": {
            "category": {
                "type": "string",
                "description": "Product category such as 'mini_printers' or 'mini_printer_papers'.",
            },
            "product_name": {
                "type": "string",
                "description": "Name of the product to check price for.",
            },
        },
        "required": ["category", "product_name"],
    },
}

availability_function = {
    "name": "check_product_availability",
    "description": (
        "Check availability of products in a category. "
        "Returns a list of {name, brand, price, type, availability}."
    ),
    "parameters": {
        "type": "object",
        "properties": {
            "category": {
                "type": "string",
                "description": "Category of products to check (e.g., 'mini_printers').",
            },
            "filters": {
                "type": "object",
                "description": "Optional filters like brand or type.",
            },
        },
        "required": ["category"],
    },
}

book_function = {
    "name": "place_order",
    "description": (
        "Place an order using an index for the chosen product. "
        "Generates a unique receipt file customerName_{orderNumber}.txt."
    ),
    "parameters": {
        "type": "object",
        "properties": {
            "category": {
                "type": "string",
                "description": "Product category (e.g., 'mini_printers').",
            },
            "product_index": {
                "type": "string",
                "description": "1-based index of selected product from availability list.",
            },
            "quantity": {
                "type": "string",
                "description": "Quantity to order.",
            },
            "customer_name": {
                "type": "string",
                "description": "Customer's full name.",
            },
            "contact_info": {
                "type": "string",
                "description": "Customer's contact information (email or phone).",
            },
        },
        "required": ["category", "product_index", "quantity", "customer_name", "contact_info"],
    },
}

report_function = {
    "name": "generate_report",
    "description": (
        "Generates a summary report of ALL orders in order_summary.txt."
    ),
    "parameters": {
        "type": "object",
        "properties": {},
        "required": [],
    },
}

tools = [
    {"type": "function", "function": price_function},
    {"type": "function", "function": availability_function},
    {"type": "function", "function": book_function},
    {"type": "function", "function": report_function},
]


In [ ]:
###############################################################################
# 5) Handle Tool Calls for Art-Tech Store
###############################################################################
def handle_tool_call(message):
    """
    The LLM can request to call a function in 'tools'. We parse the JSON arguments
    and run the corresponding Python function. Then we return a 'tool' message with the result.
    """
    tool_call = message.tool_calls[0]
    fn_name   = tool_call.function.name
    args      = json.loads(tool_call.function.arguments)

    if fn_name == "get_product_price":
        category = args.get("category")
        product_name = args.get("product_name")
        products = product_availability.get(category.lower(), [])
        price = "Not found"
        for p in products:
            if p["name"].lower() == product_name.lower():
                price = p["price"]
                break
        response_content = {"category": category, "product_name": product_name, "price": price}

    elif fn_name == "check_product_availability":
        category = args.get("category")
        filters = args.get("filters", {})
        products = check_product_availability(category, filters)
        response_content = {"category": category, "availability": products}

    elif fn_name == "place_order":
        category      = args.get("category")
        product_index = args.get("product_index")
        quantity      = args.get("quantity")
        customer_name = args.get("customer_name")
        contact_info  = args.get("contact_info")

        confirmation = place_order(category, product_index, quantity, customer_name, contact_info)
        response_content = {
            "category": category,
            "product_index": product_index,
            "quantity": quantity,
            "customer_name": customer_name,
            "contact_info": contact_info,
            "confirmation": confirmation,
        }

    elif fn_name == "generate_report":
        msg = generate_report()
        response_content = {"report": msg}

    else:
        response_content = {"error": f"Unknown tool: {fn_name}"}

    return {
        "role": "tool",
        "content": json.dumps(response_content),
        "tool_call_id": tool_call.id,
    }, args


In [ ]:
###############################################################################
# 6) Main Chat Function for Art-Tech Store
###############################################################################
def chat(message, history):
    """
    The main chat loop that handles the conversation with the user,
    passing 'tools' definitions to the LLM for function calling.
    """
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    try:
        response = openai.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=tools
        )

        # If the LLM requests a function call, handle it
        while response.choices[0].finish_reason == "tool_calls":
            msg = response.choices[0].message
            print(f"[INFO] Tool call requested: {msg.tool_calls[0]}")
            tool_response, tool_args = handle_tool_call(msg)
            print(f"[INFO] Tool response: {tool_response}")

            # Add both the LLM's request and our tool response to the conversation
            messages.append(msg)
            messages.append(tool_response)

            # Re-send updated conversation to get final or next step
            response = openai.chat.completions.create(
                model=MODEL,
                messages=messages
            )

        # Return normal text response (finish_reason = "stop")
        return response.choices[0].message.content

    except Exception as e:
        print(f"[ERROR] {e}")
        return "I'm sorry, something went wrong while processing your request."


In [ ]:
###############################################################################
# 7) Launch Gradio
###############################################################################
gr.ChatInterface(fn=chat, type="messages").launch()